# Introduction

Source: http://insideairbnb.com/get-the-data.html <br />
Data from: Rio de Janeiro, Rio de Janeiro, Brazil <br />

**Note:** <br />
* The listings dataset wasn't considered in this analysis as the filesize would exceed Github's max filesize limit <br />
* The reviews dataset had to be processed in order to fit the max filesize from Github
<br />

**Reviews_reduced creation script** 
<br />
<code>reviews.drop(['id', 'reviewer_name', 'reviewer_id'], axis = 1, inplace = True)
reviews.to_csv(data_filepath + 'reviews_reduced.csv')<code>

In [1]:
# Importing relevant liberaries for EDA
import pandas as pd
import numpy as np
import seaborn as sns
import os
import scipy

# Modules for data processing
import nltk

nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

import re
import ast

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leonardoyamaguishi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leonardoyamaguishi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leonardoyamaguishi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leonardoyamaguishi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Library to calculate distance between two geo-locations
# Source: https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b
!pip install haversine
import haversine as hs

In [3]:
os.path.abspath('')[:-9]

'/Users/leonardoyamaguishi/Documents/Udacity/data_science_capstone_project/'

In [4]:
# Reading the datasets
data_filepath = os.path.abspath('')[:-9] + '/data/'

# calendar = pd.read_csv(data_filepath + 'calendar.csv') removed from the analysis, as the file is too heavy for Git
listings = diet = pd.read_csv(data_filepath + 'listings.csv')

reviews = pd.read_csv(data_filepath + 'reviews_reduced.csv')

neighbourhood_hdi = pd.read_csv(data_filepath + 'neighbourhood_hdi.csv',
                                sep = ';',
                                decimal = ',',
                                encoding = 'utf-8')

/Users/leonardoyamaguishi/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Storing the geo-locations of metro statios in Rio de Janeiro
# Data obatined from Google Maps
# https://www.google.com/maps/search/estações+de+metro+rio+de+janeiros/@-22.8916528,-43.3403456,12z/data=!3m1!4b1
# Collected in 24/01/2022

metro_stations = {
    'central': (-22.9049171010074, -43.19083837044949),
    'gloria': (-22.920550514351582, -43.17648410807313),
    'estacio': (-22.913496579807873, -43.2064926626525),
    'botafogo': (-22.951082017707268, -43.18411168513209),
    'pavuna': (-22.806666225646197, -43.36493532847317),
    'cardeal_arcoverde' : (-22.96463512369287, -43.180656232771874),
    'iraja' : (-22.84796020759417, -43.32343657311362),
    'uruguaiana' : (-22.902940565469503, -43.18149436629887),
    'cinelandia': (-22.911063930197916, -43.175678735438794),
    'nova_america':(-22.879278506956446, -43.27187285300277),
    'coelho_neto': (-22.83150705338417, -43.34329456463904),
    'flamengo' : (-22.937231062820434, -43.17823192129637),
    'cidade_nova' : (-22.908770113484287, -43.20630803329488),
    'uruguai' : (-22.931835141573387, -43.23964451197971),
    'vicente_carvalho' : (-22.85399039798257, -43.31315765809618),
    'catete' : (-22.925787988052296, -43.17657983867064),
    'acesso_a_cardeal_arcoverde' : (-22.964675568740653, -43.18054869617871),
    'cantagalo' : (-22.976672769596224, -43.19385233439477),
    'afonso_pena' : (-22.91829820485412, -43.21782218194045),
    'general_osorio' : (-22.984766408179055, -43.19716911734364)
}

In [6]:
# Storing the geo-locations of metro statios in Rio de Janeiro
# Data obatined from Google Maps
# https://www.google.com/maps/search/estações+de+metro+rio+de+janeiros/@-22.8916528,-43.3403456,12z/data=!3m1!4b1
# Collected in 24/01/2022

metro_stations_google_score = {
    'central': 3.7,
    'gloria': 4.4,
    'estacio': 4.0,
    'botafogo': 4.0,
    'pavuna': 3.5,
    'cardeal_arcoverde' : 4.3,
    'iraja' : 3.8,
    'uruguaiana' : 4.0,
    'cinelandia': 4.4,
    'nova_america': 4.2,
    'coelho_neto': 3.7,
    'flamengo' : 4.3,
    'cidade_nova' : 4.2,
    'uruguai' : 4.4,
    'vicente_carvalho' : 4.1,
    'catete' : 4.3,
    'acesso_a_cardeal_arcoverde' : 4.3, # No score, but seems to be the same as cardeal_arcoverde
    'cantagalo' : 4.4,
    'afonso_pena' : 4.1,
    'general_osorio': 4.1 
}

# ETL - Extract, transform, load

## Neighbourhood HDI ETL
**Download source:** https://www.data.rio/documents/58186e41a2ad410f9099af99e46366fd/about <br />
**Data source:** Dados básicos: IBGE-microdados dos Censos Demográficos 1991 e 2000.<br />
**Note:** <br/>
* To generate the csv file in the data folder, some formatting on Excel was performed translating the columns and removing columns not considered in this analysis.<br/>
* IBGE - Brazilian Institute of Geography and Statistics.<br/>
* Some neighbourhood names were adjusted in order to allow the retrieval of the HDI based on the neighbourhood names, some small research was done in order to increase the relieability of the data.

## Useful functions for neighbourhood_hdi ETL

In [7]:
def unpack_double_assessment(neighbourhood_hdi = neighbourhood_hdi):
    '''
    INPUT:
    neighbourhood_hdi - (DataFrame) in which more than one neighbourhood was assessed at once
    Note: In such cases, the different neighbourhoods were listed in the same row, separated by ","
    
    OUTPUT:
    unpacked_neighbourhood_hdi - (DataFrame) with unpacked neighbourhoods (the rankings will be repeated)
    '''
    delist_after_treatment = []
    
    # Loops through indexes and neighbourhood names
    for index, neighbourhood in enumerate(neighbourhood_hdi.neighbourhood):
        
        # If the neighbourhood column has ','
        # means that more than one neighbourhood was assessed for the respective row
        if neighbourhood_hdi['neighbourhood'][index].find(',') != -1:
            
            # Lists indexes to drop after unpacking
            delist_after_treatment.append(index)
            
            # Lists the names inside the neighbourhood name column
            names_to_unpack = neighbourhood_hdi.neighbourhood[index].split(',')
            
            # Copies the entire row as a template, as the only change in the unpacked values will be the name
            template_series = neighbourhood_hdi.iloc[index]

            # Loops through the names to unpack
            for name in names_to_unpack:
                # Duplicates the template series
                series_to_append = template_series
                # Changes the neighbourhood name from the template series
                series_to_append['neighbourhood'] = name
                # Appends the new unpacked neighbourhood
                neighbourhood_hdi = neighbourhood_hdi.append(series_to_append, ignore_index = True)
    
    # After all iterations, the listed indexes to drop are dropped
    unpacked_neighbourhood_hdi = neighbourhood_hdi.drop(delist_after_treatment, axis = 0)
    
    return unpacked_neighbourhood_hdi

In [8]:
def standardize_names(txt):
    '''
    INPUTS:
    txt - (string) in which the characters will be converted to lower case and the spaces will be removed
    OUTPUTS:
    standardized_txt - lower case (string) without spaces
    '''
    return str(txt.lower().replace(' ', ''))

In [9]:
def neighbourhood_hdi_update_keys(neighbourhood_hdi = neighbourhood_hdi):
    '''
    INPUTS:
    neighbourhood_hdi - (DataFrame) to have the keys created
    
    OUTPUTS:
    neighbourhood_hdi - (DataFrame) with updated keys according to their HDI rank
    '''
    neighbourhood_hdi.sort_values(by = 'human_development_index', ascending = False, inplace = True)
    neighbourhood_hdi['neighbourhood_key'] = list(neighbourhood_hdi.index)
    
    return neighbourhood_hdi

## neighbourhood_hdi ETL

In [10]:
neighbourhood_hdi.head(n = 2)

,hdi_ranking,neighbourhood,per_capita_income_2000,longevity_index,education_index,income_index,human_development_index
0,1,Gávea,2139.559275,0.924139,0.987167,1.0,0.970435
1,2,Leblon,2441.279121,0.907799,0.993403,1.0,0.967067


In [11]:
# In this IBGE study, some neighbourhoods were assessed together
# In such cases the different neighbourhoods are listed in the same row, separated by ","
print(neighbourhood_hdi.iloc[115])

hdi_ranking                                                   116
neighbourhood              Camorim, Vargem Pequena, Vargem Grande
per_capita_income_2000                                 279.093541
longevity_index                                          0.688297
education_index                                          0.836278
income_index                                             0.712834
human_development_index                                  0.745803
Name: 115, dtype: object


In [12]:
# Neighbourhood names unique values check
neighbourhood_unnique = neighbourhood_hdi['neighbourhood'].nunique() == neighbourhood_hdi['neighbourhood'].shape[0]
print('Neighbourhood column has only unique values: ' + str(neighbourhood_unnique))

Neighbourhood column has only unique values: True


In [13]:
# Dropping the HDI Ranking as it can be redundant given the fact that the DataFrame can be sorted when necessary
neighbourhood_hdi.drop('hdi_ranking', axis = 1, inplace = True)

# Unpacking double assessments 
neighbourhood_hdi = unpack_double_assessment(neighbourhood_hdi)

# Standardizing neighbourhood names
neighbourhood_hdi['neighbourhood'] = neighbourhood_hdi['neighbourhood'].apply(standardize_names)

# Updating neighbourhood keys
neighbourhood_hdi = neighbourhood_hdi_update_keys(neighbourhood_hdi)

<ipython-input-7-a3cfc2665ea3>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series_to_append['neighbourhood'] = name


In [14]:
neighbourhood_hdi.head(n = 2)

,neighbourhood,per_capita_income_2000,longevity_index,education_index,income_index,human_development_index,neighbourhood_key
0,gávea,2139.559275,0.924139,0.987167,1.0,0.970435,0
1,leblon,2441.279121,0.907799,0.993403,1.0,0.967067,1


## Useful functions for Listings ETL

In [15]:
def tokenize(text):
    '''
    INPUTS:
    text - a string to be tokenized
    OUTPUTS:
    clean_tokens - (list) of clean tokens
    '''
    
    # Removing stop words 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenizing the text
    tokenized_text = word_tokenize(text)
    
    # Removing stopwords - assuming that all amenities are listed in english
    tokenized_text = [w for w in tokenized_text if w not in stopwords.words("english")]
    
    # If the text is not a stop word, it should be tokenized
    if tokenized_text != []:
        # Starts the lemmatizer
        lemmatizer = WordNetLemmatizer()

        # Creates a list of lemmatized tokens
        clean_tok = lemmatizer.lemmatize(tokenized_text[0]).lower().strip()
    else:
        clean_tok = ''
        
    return clean_tok

In [16]:
def token_list_to_string(token_list):
    '''
    INPUTS:
    token_list - (list) of tokens
    
    OUTPUTS:
    token_list_string - (str) in which the tokens are separated by ' '
    '''
    token_list_string = ''
    
    n = 0
    
    for n in range (0, len(token_list)):
        
        # On the start of the loop, no space is necessary
        if n != 0:
            token_list_string += ' '
            
        # adds on the token_list_string the a unique token (can be a compound noun)
        token_list_string += token_list[n]
        
    return token_list_string

In [17]:
def literal_to_list_and_tokenize(list_string):
    '''
    INPUTS:
    listings_series - (Series) with a column with literal list strings composed of strings to be tokenized
    
    OUTPUTS:
    token_list_as_string - (str) in which the tokens are separated by ' ' (adapted for CountVectorizer)
    '''
    
    # Converts literal list to list
    converted_list = ast.literal_eval(list_string)
          
    for i in range (0, len(converted_list)):
        
        # As each item in the list is a item that could be a compound noun
        # The objective is to tokenize each element of the compound noun and join then with ' '
        # Increasing the chance of finding matches throughout the dataset
        
        compound_noun = converted_list[i].split()
        
        generated_word = ''
        
        for l in range(0, len(compound_noun)):
            tokenized_element = tokenize(str(compound_noun[l]).lower())
            
            # Tokenized_elemnt is '' when the given word is a stop word
            if tokenized_element != '':
                
                # If not a stop word, check if it's the first element of the possible compound noun
                if l != 0:
                    generated_word += ' '
                    
                # If compound_noun has length above 1, it's a compound noun
                generated_word += tokenize(str(compound_noun[l]).lower())
        
        # Adds the compound noun to the converted list
        converted_list[i] = generated_word

    # Convert the converted_list of tokenized compound nouns to a string separating them with ' '
    token_list_as_string = token_list_to_string(converted_list)
    
    return token_list_as_string #ASSESSMENT

In [18]:
def list_nan_columns(df):
    '''
    INPUT:
    df - (DataFrame) in which the columns composed entirely of NaN values will be listed
    OUTPUT:
    nan_columns_list - (list) of columns composed entirely of NaN values
    '''
    
    nan_columns_list = []
    
    for column in df.columns:
        if df[column].isna().sum() == df.shape[0]:
            nan_columns_list.append(column)
            
    return nan_columns_list

In [19]:
def compute_walking_dist_stations(listings = listings, metro_stations = metro_stations, walking_dist = 1):
    '''
    INPUTS:
    listings - (DataFrame) with AirBnb listings with latitude and longitude columns
    metro_stations - (Dictionary) with the name of the metro stations as keys and (tuples)
    of their respective latitude and longitude
    
    OUTPUTS:
    nearest_stations_walking_dist - (list) of the nearest stations within a distance of 1 km
    '''
    
    # Prepares the lists
    nearest_stations_walking_dist = []
    
    # Loops through the latitudes and longitudes of the AirBnBs
    for latitude, longitude in zip(listings.latitude, listings.longitude):
        
        # Distance place holder
        nearest_station = 'NONE'
        nearest_station_dist = 999999
        
        # Loops through the metro_stations dictionary
        for station in metro_stations:
            
            # Computes distance according to the Haversine distance
            dist = hs.haversine((latitude, longitude), metro_stations[station])
            
            # Updates the variables if a nearer metro station is found
            if dist < nearest_station_dist:
                nearest_station = station
                nearest_station_dist = dist
        
        # Updates the lists
        if dist <= walking_dist:
            nearest_stations_walking_dist.append(station)
        else:
            nearest_stations_walking_dist.append('NONE')
        
        station_score_list = assign_station_score(nearest_stations_walking_dist, metro_stations_google_score)
    return nearest_stations_walking_dist, station_score_list

In [20]:
def listings_assign_keys(listings = listings, neighbourhood_hdi = neighbourhood_hdi):
    '''
    INPUTS: 
    listings - (DataFrame) with 'neighbourhood' column that matches the neighbourhoods in neighbourhood_hdi
    neighbourhood_hdi - (DataFrame) with 'neighbourhood' and 'neighbourhood_key' columns
    
    OUTPUTS:
    listings - (DataFrame) with new or updated 'neighrbouhood_key' column matching neighbourhood_hdi
    '''
    
    # Updating neighbourhood_hdi keys
    neighbourhood_hdi = neighbourhood_hdi_update_keys(neighbourhood_hdi)
    
    # Creates a dictionary that connects the neighbourhood name to its respective key
    name_key_dict = {}

    for name, key in zip(neighbourhood_hdi.neighbourhood, neighbourhood_hdi.neighbourhood_key):
        name_key_dict[name] = key

    listings_keys = []

    # Assigns the secondary key to listings 'neighbourhood_key' column according to its 'neighbourhood'
    for name in listings.neighbourhood:
        listings_keys.append(name_key_dict[name])

    listings['neighbourhood_key'] = listings_keys
    
    return listings



In [21]:
def assign_station_score(stations_list, metro_stations_google_score = metro_stations_google_score):
    '''
    INPUTS:
    stations_list - (list) of metro stations mapped in the dictionaries declared in the start of the notebook
    metro_stations_google_score - (dictionary) that links the metro stations (keys) with their respective
    google scores
    
    OUTPUTS:
    station_score_list - (list) of google scores in the same order as the stations_list

    '''
    
    station_score_list = []
    
    for station in stations_list:
        if station != "NONE":
            station_score_list.append(metro_stations_google_score[station])
        else: 
            # -1 is assigned to the case NONE
            station_score_list.append(-1)
            
    return station_score_list

In [22]:
def summarize_bathroom_string(listings = listings):
    '''
    INPUTS:
    listings - DataFrame with bathrooms_text, in which "shared" indicated shared bathrooms
    
    OUTPUTS:
    shared_bathroom - (list) with 1 for True and 0 for False
    bathroom_qty - (list) quantity of bathroom for the respective listing
    '''
    shared_bathroom = []
    bathroom_qty = []

    for txt in listings.bathrooms_text:
        
        # Retrieving the bathroom quantity
        try:
            bathroom_qty.append(float(str(txt)[0:(str(txt).lower().find(' '))]))
        except:
            # Some strings do not have quantity, therefore are considered as nan
            bathroom_qty.append(np.nan)
            
        # If the bathrooms_text has shared in the string, it indicates a shared bathroom
        if str(txt).lower().find('shared') != -1:
            shared_bathroom.append(1)
        else:
            shared_bathroom.append(0)
            
    return bathroom_qty, shared_bathroom

In [23]:
def standardize_boolean_columns(txt):
    '''
    INPUTS:
    txt - (string or nan) with "t" for True and "f" for False
    OUTPUTS:
    1 - (int) for "t"
    0 - (int) for "f" and nan
    '''
    if txt == 't': # Case t (True)
        return 1
    else: # Cases f (False) and nan
        return 0

## Listings ETL

In [24]:
listings.head(n = 2)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20211224070558,2021-12-25,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Discounts for long term stays. <br />- Large b...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,...,4.9,4.76,4.66,NaN,f,1,1,0,0,1.92
1,24480,https://www.airbnb.com/rooms/24480,20211224070558,2021-12-25,"Nice and cozy near Ipanema Beach, w/ home office","My studio is located in the best of Ipanema, t...","The beach, the lagoon, Ipanema is a great loca...",https://a0.muscache.com/pictures/11955612/b28e...,99249,https://www.airbnb.com/users/show/99249,...,4.9,4.97,4.58,NaN,f,1,1,0,0,0.62


In [25]:
delisted_columns = [
                            'name',
                            'description',
                            'neighborhood_overview',
                            'listing_url',
                            'scrape_id',
                            'last_scraped',
                            'picture_url',
                            'host_picture_url',
                            'host_url',
                            'host_location',
                            'host_thumbnail_url',
                            'host_total_listings_count',
                            'host_name',
                            'host_about',
                            'host_neighbourhood',
                            'host_verifications',
                            'property_type', 
                            'neighbourhood',
                            'availability_30',
                            'availability_60',
                            'availability_90',
                            'availability_365',
                            'instant_bookable',
                            'calculated_host_listings_count',
                            'calculated_host_listings_count_entire_homes',
                            'calculated_host_listings_count_private_rooms',
                            'calculated_host_listings_count_shared_rooms',
                            'minimum_nights',
                            'maximum_nights',
                            'minimum_minimum_nights',
                            'maximum_minimum_nights',
                            'minimum_maximum_nights',
                            'maximum_maximum_nights',
                            'has_availability',
                            'calendar_last_scraped',
                            'number_of_reviews',
                            'first_review',
                            'last_review',
                            'host_acceptance_rate',
                            'host_listings_count',
                            'number_of_reviews_l30d'
                            ]

In [26]:
# Hypothesis: listings.id is a column of primary key values
listings.id.nunique() == listings.shape[0]

True

In [27]:
# Evaluating possible redundant column
print(str((listings.host_listings_count == listings.host_total_listings_count).sum() == listings.shape[0]) + '\n')

diff_list = []
for control, test in zip(listings.host_listings_count, listings.host_total_listings_count):
    if control != test:
        diff_list.append((control, test))
        
print(diff_list)

# Conclusion: any of the 2 evaluated columnns can be dropped

False

[(nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan), (nan, nan)]


In [28]:
# Checking and dropping columns composed entirely of NaN values
listings_columns_to_drop = list_nan_columns(listings)
listings.drop(listings_columns_to_drop, axis = 1, inplace = True)

# Dropping delisted columns, these columns were dropped as they were out of the scope of the analysis
# given their definition
listings.drop(delisted_columns, axis = 1, inplace = True)

# Summarizing and replacing the bathrooms_text column
bathroom_qty, shared_bathroom = summarize_bathroom_string(listings)
listings['bathroom_qty'] = bathroom_qty
listings['shared_bathroom'] = shared_bathroom
listings.drop(['bathrooms_text'], axis = 1, inplace = True)

# Standardizing boolean columns
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].apply(standardize_boolean_columns)
listings['host_identity_verified'] = listings['host_identity_verified'].apply(standardize_boolean_columns)
listings['host_is_superhost'] = listings['host_is_superhost'].apply(standardize_boolean_columns)

# Converting literal lists to token strings
listings['amenities'] = listings['amenities'].apply(literal_to_list_and_tokenize)

# Computing the nearest stations
walking_dist_stations, walking_dist_stations_scores = compute_walking_dist_stations(listings, metro_stations)

# Appending in the DataFrame
listings['walking_dist_stations'] = walking_dist_stations
listings['walking_dist_stations_scores'] = walking_dist_stations_scores

# Dropping the latitude and longitude columns
listings.drop(['latitude', 'longitude'], axis = 1, inplace = True)

# Standardizing neighbourhood names
listings['neighbourhood'] = listings.neighbourhood_cleansed.apply(standardize_names)

# Dropping the non standardized column
listings.drop(['neighbourhood_cleansed'], axis = 1, inplace = True)

In [29]:
# Assessing the neighbourhoods that could not be retrieved from the IBGE data

assess = []
for neighbourhood in listings.neighbourhood:
    if neighbourhood not in list(neighbourhood_hdi['neighbourhood']):
        assess.append(neighbourhood)

set(assess)

{'gericinó'}

In [30]:
# For 'gericinó'the missing data will be the average of the hdi table

# Creating a series with the neighbourhood data
gericino_case = pd.Series(data = 'gericinó', index = ['neighbourhood'])

# Calculating the mean for the numeric columns
gericino_series = neighbourhood_hdi.mean().append(gericino_case)

# Reindexing the series to append in neighbourhood_hdi
gericino_series.reindex(['neighbourhood',
                        'per_capita_income_2000',
                        'longevity_index', 
                        'education_index', 
                        'income_index', 
                        'human_development_index'])

# Appending new data 
neighbourhood_hdi = neighbourhood_hdi.append(gericino_series, ignore_index = True)

# Updating neighbourhood_hdi keys
neighbourhood_hdi = neighbourhood_hdi_update_keys(neighbourhood_hdi)

# Assigning keys to the neighbourhoods in listings.neighbourhoods
listings = listings_assign_keys(listings , neighbourhood_hdi)

In [31]:
# Expading the listings DataFrame
listings_expanded = listings.join(neighbourhood_hdi, on = 'neighbourhood_key', how = 'left', lsuffix = '_hdi')

# Dropping the key columns and neighbourhood names
listings_expanded.drop(['neighbourhood_key',
                        'neighbourhood_key_hdi',
                        'neighbourhood_hdi',
                        'neighbourhood']
                       , axis = 1, inplace = True)

In [32]:
vect = CountVectorizer(tokenizer=None)

# Getting the counts for each token
X = vect.fit_transform(listings_expanded['amenities'])

# Converting to dense matrix to save
X_dense = scipy.sparse.csr_matrix.todense(X)

# Inverting the dictionary from {key : value} to {value : key}
# Source: https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping
matrix_vocabulary = {value: key for key, value in vect.vocabulary_.items()}

# Retrieving the columns from the CountVectorizer matrix according to the generated vocabulary
matrix_columns = []
for i in range(0, len(matrix_vocabulary)):
    matrix_columns.append('amn_' + matrix_vocabulary[i])
    
# Converting the dense matrix to DataFrame
amenities_dataframe = pd.DataFrame(X_dense, columns = matrix_columns, index = listings_expanded.index)

# Dropping amenities from listings_expanded
listings_expanded.drop(['amenities'], axis = 1, inplace = True)

# Saving the processed DataFrames as csv
amenities_dataframe.to_csv(data_filepath + 'amenities_processed.csv')
listings_expanded.to_csv(data_filepath + 'listings_processed.csv')

In [33]:
listings_expanded.head(n = 2)

,id,host_id,host_since,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,room_type,accommodates,...,reviews_per_month,bathroom_qty,shared_bathroom,walking_dist_stations,walking_dist_stations_scores,per_capita_income_2000,longevity_index,education_index,income_index,human_development_index
0,17878,68997,2010-01-08,within an hour,100%,0,1,1,Entire home/apt,5,...,1.92,1.0,0,NONE,-1.0,1623.415521,0.879608,0.989888,1.0,0.956499
1,24480,99249,2010-03-26,a few days or more,100%,0,1,1,Entire home/apt,2,...,0.62,1.0,0,general_osorio,4.1,2465.445144,0.894594,0.991860,1.0,0.962151


In [34]:
amenities_dataframe.head(n = 2)

,amn_02,amn_04,amn_10,amn_100,amn_101,amn_102,amn_104,amn_105,amn_107,amn_10g,...,amn_xampu,amn_xbox,amn_xiaomi,amn_xiomi,amn_xxx,amn_yamaha,amn_yamasterol,amn_year,amn_youtube,amn_yp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
